<img src="/home/ahmedunix/data_Science_Work/Machine_Learning/16_Principal_Component_Analysis_(PCA)/excercise/excercise_PCA.png">

In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import tensorflow as tf, keras
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import k_means
from sklearn.decomposition import PCA

2024-10-04 17:53:34.123206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 17:53:34.157633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 17:53:34.167688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 17:53:36.111365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
import kaggle as kaggle

In [24]:
!chmod 600 /home/ahmedunix/.config/kaggle/kaggle.json

In [18]:
!kaggle datasets download -d fedesoriano/heart-failure-prediction

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
License(s): ODbL-1.0
  0%|                                               | 0.00/8.56k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 8.56k/8.56k [00:00<00:00, 1.26MB/s]


In [19]:
!unzip heart-failure-prediction.zip -d heart_failure_dataset

Archive:  heart-failure-prediction.zip
  inflating: heart_failure_dataset/heart.csv  


In [38]:
df = pd.read_csv("/home/ahmedunix/data_Science_Work/Machine_Learning/16_Principal_Component_Analysis_(PCA)/excercise/heart_failure_dataset/heart.csv")

In [39]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [41]:
df.head(n=10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [42]:
df['Age_z_score'] = (df['Age'] - df['Age'].mean()) / df['Age'].std()

In [43]:
df['RestingBP_z_score'] = (df['RestingBP'] - df['RestingBP'].mean()) / df['RestingBP'].std()

In [44]:
df['Cholesterol_z_score'] = (df['Cholesterol'] - df['Cholesterol'].mean()) / df['Cholesterol'].std()

In [46]:
df['MaxHR_z_score'] = (df['MaxHR'] - df['MaxHR'].mean()) / df['MaxHR'].std()

In [48]:
df['Oldpeak_z_score'] = (df['Oldpeak'] - df['Oldpeak'].mean()) / df['Oldpeak'].std()

In [49]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,1.382175,-0.831979
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,0.753746,0.105606
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-1.131539,0.574398
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979


In [54]:
df['Outlier'] = ((df['Age_z_score'] >= 3) | (df['RestingBP_z_score'] >= 3)
                 | (df['Cholesterol_z_score'] >= 3) | (df['MaxHR_z_score'] >= 3)
                 | (df['Oldpeak_z_score'] >= 3) | (df['Age_z_score'] <= -3)
                 | (df['RestingBP_z_score'] <= -3) | (df['Cholesterol_z_score'] <= -3)
                 | (df['MaxHR_z_score'] <= -3) | (df['Oldpeak_z_score'] <= -3))

In [64]:
df[df['Outlier'] == True].shape

(19, 18)

In [61]:
df_no_outliers = df[df['Outlier']==False].copy()

In [62]:
df_no_outliers

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score,Outlier
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,1.382175,-0.831979,False
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,0.753746,0.105606,False
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979,False
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-1.131539,0.574398,False
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,-0.902283,-1.209697,0.596068,-0.188897,0.293123,False
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,1.536064,0.626736,-0.053020,0.164595,2.355810,False
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,0.369898,-0.129442,-0.619830,-0.856602,0.293123,False
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0.369898,-0.129442,0.340090,1.460728,-0.831979,False


In [65]:
df_no_outliers.drop(columns=['Age_z_score','RestingBP_z_score','Cholesterol_z_score','MaxHR_z_score','Oldpeak_z_score','Outlier'],inplace=True)

In [67]:
df_no_outliers

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [68]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

In [72]:
ohe_transform = ohe.fit_transform(df_no_outliers[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']])

In [73]:
ohe_transform

,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
914,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
915,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
916,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [74]:
df_ohe = pd.concat([df_no_outliers,ohe_transform], axis='columns').drop(
    columns=['Sex','Sex_M','ChestPainType','ChestPainType_TA','RestingECG',
             'RestingECG_ST','ExerciseAngina','ExerciseAngina_N','ST_Slope','ST_Slope_Up'])

In [75]:
df_ohe

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,RestingECG_LVH,RestingECG_Normal,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat
0,40,140,289,0,172,0.0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,49,160,180,0,156,1.0,1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,37,130,283,0,98,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48,138,214,0,108,1.5,1,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,54,150,195,0,122,0.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
914,68,144,193,1,141,3.4,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
915,57,130,131,0,115,1.2,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
916,57,130,236,0,174,0.0,1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [76]:
x = df_ohe.drop(columns=['HeartDisease'])

In [78]:
y = df_ohe.HeartDisease

In [83]:
scaler = StandardScaler().set_output(transform='pandas')

In [84]:
x_scaled = scaler.fit_transform(x)

In [85]:
x_scaled

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,RestingECG_LVH,RestingECG_Normal,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat
0,-1.428154,0.465900,0.849636,-0.550362,1.384320,-0.855469,-0.515943,-1.077524,2.063325,-0.534905,-0.503821,0.809702,-0.822945,-0.260184,-0.998888
1,-0.475855,1.634714,-0.168122,-0.550362,0.752973,0.137516,1.938199,-1.077524,-0.484655,1.869492,-0.503821,0.809702,-0.822945,-0.260184,1.001113
2,-1.745588,-0.118507,0.793612,-0.550362,-1.535661,-0.855469,-0.515943,-1.077524,2.063325,-0.534905,-0.503821,-1.235023,-0.822945,-0.260184,-0.998888
3,-0.581666,0.349019,0.149344,-0.550362,-1.141069,0.634008,1.938199,0.928054,-0.484655,-0.534905,-0.503821,0.809702,1.215148,-0.260184,1.001113
4,0.053200,1.050307,-0.028064,-0.550362,-0.588640,-0.855469,-0.515943,-1.077524,-0.484655,1.869492,-0.503821,0.809702,-0.822945,-0.260184,-0.998888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.899099,-1.287320,0.616205,-0.550362,-0.194048,0.336112,-0.515943,-1.077524,-0.484655,-0.534905,-0.503821,0.809702,-0.822945,-0.260184,1.001113
914,1.534554,0.699663,-0.046738,1.816985,0.161085,2.520678,-0.515943,0.928054,-0.484655,-0.534905,-0.503821,0.809702,-0.822945,-0.260184,1.001113
915,0.370633,-0.118507,-0.625646,-0.550362,-0.864854,0.336112,-0.515943,0.928054,-0.484655,-0.534905,-0.503821,0.809702,1.215148,-0.260184,1.001113
916,0.370633,-0.118507,0.354763,-0.550362,1.463238,-0.855469,1.938199,-1.077524,2.063325,-0.534905,1.984833,-1.235023,-0.822945,-0.260184,1.001113


In [86]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,random_state=20,test_size=0.2)

In [91]:
model_svm = SVC()
model_svm.fit(x_train,y_train)
model_svm.score(x_test,y_test)

0.8944444444444445

In [94]:
model_logistic_regression = LogisticRegression()
model_logistic_regression.fit(x_train,y_train)
model_logistic_regression.score(x_test,y_test)

0.9166666666666666

In [97]:
model_random_forest = RandomForestClassifier()
model_random_forest.fit(x_train,y_train)
model_random_forest.score(x_test,y_test)

0.9111111111111111

In [192]:
pca = PCA(n_components=6)

In [193]:
x_pca = pca.fit_transform(x)

In [194]:
x_pca

array([[ 9.31294527e+01,  2.96782587e+01,  1.09495058e+01,
        -9.16631305e+00, -6.57857117e-01, -3.23141831e-01],
       [-1.63381696e+01,  1.48066289e+01,  3.10882747e+01,
        -6.20368351e+00,  3.64599076e-02, -7.47622943e-01],
       [ 8.26699370e+01, -3.89114403e+01, -1.47875970e+01,
        -2.11461710e+01, -1.08423217e+00, -2.87180165e-01],
       ...,
       [-6.82263522e+01, -1.76984976e+01, -4.33443820e+00,
         3.89255662e-01,  4.17585468e-01,  1.26246805e-01],
       [ 4.00272248e+01,  3.34726805e+01,  5.36107672e+00,
         9.19087378e+00, -7.79856138e-01,  5.03755697e-01],
       [-2.06126669e+01,  3.76161290e+01,  1.21566507e+01,
        -1.09842984e+01, -4.77428433e-01, -4.10011349e-01]])

In [195]:
x_pca.shape

(899, 6)

In [196]:
pca.explained_variance_ratio_

array([9.21124929e-01, 5.06478614e-02, 2.25812128e-02, 5.43667339e-03,
       8.22337269e-05, 2.60787877e-05])

In [197]:
pca.n_components_

6

In [198]:
x_train_pca,x_test_pca,y_train_pca,y_test_pca = train_test_split(x_pca,y,random_state=20,test_size=0.2)

In [199]:
model_svm = SVC()
model_svm.fit(x_train_pca,y_train_pca)
model_svm.score(x_test_pca,y_test_pca)

0.7666666666666667

In [200]:
model_logistic_regression = LogisticRegression(max_iter=500)
model_logistic_regression.fit(x_train_pca,y_train_pca)
model_logistic_regression.score(x_test_pca,y_test_pca)

0.8777777777777778

In [201]:
model_random_forest = RandomForestClassifier()
model_random_forest.fit(x_train_pca,y_train_pca)
model_random_forest.score(x_test_pca,y_test_pca)

0.8388888888888889